# 计算用户和app的匹配程度

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 读入用户和app
dataTrain = pd.concat([pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv"),pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\test.csv")])
dataTrain  = pd.merge(dataTrain, pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv"), on='creativeID')
dataTrain = dataTrain[['label','userID','appID']]

In [3]:
dataTrain = dataTrain[dataTrain['appID']==83]

In [23]:
# 读入userSingle
dataUserSingle = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_single.csv')
dictUserSingle = {}
def getDictUserSingle(x):
    key = str(int(x[0])) + '-' + str(int(x[1])) + '-' + str(int(x[2]))
    value = eval(x[3])
    dictUserSingle[key] = value
dataUserSingle.apply(getDictUserSingle, axis=1)
print dictUserSingle['83-1-0']['4025'] #第一位是appid，第二位是single，第三位是partition

0.00571428571429


In [8]:
# 读入app_coScale
dataAppCoScale = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_coScale.csv')[['appID', 'coScale_logReci']].set_index('appID')
dictAppScore = dataAppCoScale.to_dict() #索引，为了加快速度

In [9]:
# 读取user_apps
dataUserApps = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_apps.csv', names=['userID', 'appList'])
dict_userApps = {}
def getDictUserApps(x):
     dict_userApps[x[0]] = x[1][1:-1].split(', ')
dataUserApps.apply(getDictUserApps, axis=1)
dataUserApps.head()

,userID,appList
0,1,"[357, 360, 362, 365, 375, 441, 480, 481, 482, ..."
1,9,"[538, 602, 604]"
2,10,"[357, 359, 365, 383, 480, 481, 482, 483, 484, ..."
3,12,"[480, 481, 482, 483, 484, 485, 486, 487, 488, ..."
4,14,"[362, 480, 481, 482, 483, 484, 485, 486, 487, ..."


In [38]:
# 用户与single的相似度计算函数
def getScore(list, single):
    score = 0.0
    for l in list:
        if dictUserSingle[single].has_key(l): #交集及single这边的app
            score += dictAppScore['coScale_logReci'][int(l)] * dictUserSingle[single][l]
        else: #仅user这边的app
            score += dictAppScore['coScale_logReci'][int(l)]
    return score
def cal_coScale(user, single):
    if dict_userApps.has_key(user) == False: #对于缺乏数据的用户，目前返回0
        return 0
    list0 = dict_userApps[user]
    list1 = dictUserSingle[single].keys()
    intersect = np.intersect1d(list0, list1)
    union = np.union1d(list0, list1)
    length = len(intersect)
    if(length == 0): 
        return 0
    else: 
#         return getScore(intersect, single)
        return getScore(intersect, single)/length
#         return getScore(intersect, single)/getScore(union, single)

In [39]:
def getVote(x, voteType):
    single = str(int(x[1])) + '-' + '1' + '-' + str(voteType)
    return cal_coScale(int(x[0]), single)
dataTrain['vote1'] = dataTrain[['userID','appID']].apply(getVote, args=(1,), axis=1)
dataTrain['vote0'] = dataTrain[['userID','appID']].apply(getVote, args=(0,), axis=1)

In [40]:
dataTrain['voteMinus'] = dataTrain['vote1']-dataTrain['vote0']

In [29]:
dataTrain = dataTrain[dataTrain['vote1']!=0]

In [42]:
dataTrain[dataTrain['voteMinus']<0].groupby(['label']).count()

,userID,appID,vote1,vote0,voteMinus
label,,,,,
0,1574,1574,1574,1574,1574
1,224,224,224,224,224
